Hava Durumu verisini çeken kod

In [ ]:
import requests
import pandas as pd
from datetime import datetime

API_KEY = "16ce454570964d39901204010250905"  # Buraya kendi WeatherAPI key’ini yaz.
BASE_URL = "http://api.weatherapi.com/v1/current.json"
# İncelenecek şehir listesi
cities = ["Istanbul", "Ankara", "Izmir", "Bursa", "Antalya"]

def fetch_weather(city):
    """Belirtilen şehir için anlık hava durumu verisini döndürür."""
    params = {
        "key": API_KEY,
        "q": city,
        "aqi": "no"
    }
    response = requests.get(BASE_URL, params=params)

    if response.status_code != 200:
        print(f"Hata oluştu: {city} için veri alınamadı. {response.text}")
        return None

    data = response.json()
    return {
        "city": city,
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "condition": data["current"]["condition"]["text"],
        "temp_c": data["current"]["temp_c"],
        "feelslike_c": data["current"]["feelslike_c"],
        "humidity": data["current"]["humidity"],
        "wind_kph": data["current"]["wind_kph"]
    }

def save_to_csv(data_list, filename="hava_durumu_verisi.csv"):
    df = pd.DataFrame(data_list)
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"{len(df)} şehirden hava durumu verisi '{filename}' dosyasına kaydedildi.")

# Ana veri çekme işlemi
weather_data = []
for city in cities:
    data = fetch_weather(city)
    if data:
        weather_data.append(data)

# CSV olarak kaydet
if weather_data:
    save_to_csv(weather_data)


5 şehirden hava durumu verisi 'hava_durumu_verisi.csv' dosyasına kaydedildi.


Şehire göre hava durumu verisini getiren kod

In [ ]:
import requests
from datetime import datetime

API_KEY = "16ce454570964d39901204010250905"  # Kendi API key'ini gir
city = input("🌍 Hava durumu sorgulamak istediğiniz şehri girin: ")

url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={city}&lang=tr"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    current = data["current"]
    location = data["location"]

    hava_durumu = {
        "Şehir": location["name"],
        "Ülke": location["country"],
        "Durum": current["condition"]["text"],
        "Sıcaklık (°C)": current["temp_c"],
        "Hissedilen Sıcaklık (°C)": current["feelslike_c"],
        "Nem (%)": current["humidity"],
        "Rüzgar Hızı (km/s)": current["wind_kph"],
        "Veri Saati": location["localtime"]
    }

    print("\n📊 Hava Durumu Bilgileri:")
    for etiket, bilgi in hava_durumu.items():
        print(f"{etiket}: {bilgi}")
else:
    print("🚨 Hata:", response.status_code, response.text)

🌍 Hava durumu sorgulamak istediğiniz şehri girin: ankara

📊 Hava Durumu Bilgileri:
Şehir: Ankara
Ülke: Turkey
Durum: Açık
Sıcaklık (°C): 19.3
Hissedilen Sıcaklık (°C): 19.3
Nem (%): 52
Rüzgar Hızı (km/s): 8.6
Veri Saati: 2025-05-18 00:17


Havadurumu verisini df olarak tutuan kod

In [ ]:
import pandas as pd

API_KEY = "16ce454570964d39901204010250905"  # Buraya kendi WeatherAPI key’ini yaz.
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def fetch_weather(city):
    """Belirtilen şehir için anlık hava durumu verisini döndürür."""
    params = {
        "key": API_KEY,
        "q": city,
        "aqi": "no"
    }
    response = requests.get(BASE_URL, params=params)

    if response.status_code != 200:
        print(f"Hata oluştu: {city} için veri alınamadı. {response.text}")
        return None

    data = response.json()
    return {
        "city": city,
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "condition": data["current"]["condition"]["text"],
        "temp_c": data["current"]["temp_c"],
        "feelslike_c": data["current"]["feelslike_c"],
        "humidity": data["current"]["humidity"],
        "wind_kph": data["current"]["wind_kph"]
    }

def save_to_csv(df, filename="hava_durumu_verisi.csv"):
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"{len(df)} şehirden hava durumu verisi '{filename}' dosyasına kaydedildi.")

selected_city = input("Hava durumu verisi almak istediğiniz şehri girin: ")

weather_data = fetch_weather(selected_city)

# DataFrame oluştur
if weather_data:
    df_weather = pd.DataFrame([weather_data]) # fetch_weather tek bir sözlük döndürdüğü için listeye alınır
    print("\nÇekilen Hava Durumu Verisi:")
    print(df_weather)
else:
    print(f"{selected_city} için hava durumu verisi alınamadı.")

Hava durumu verisi almak istediğiniz şehri girin: Bursa

Çekilen Hava Durumu Verisi:
    city                 time          condition  temp_c  feelslike_c  \
0  Bursa  2025-05-11 07:02:34  Light rain shower    12.1         12.6   

   humidity  wind_kph  
0        94       3.6  


In [ ]:
# weather_outfit_ml.py

import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

# 1. Hava durumu verisini WeatherAPI'den al
API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=tr"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"🚨 Hava durumu alınamadı: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    hava_durumu = {
        "Şehir": location["name"],
        "Ülke": location["country"],
        "Durum": current["condition"]["text"],
        "Sıcaklık (°C)": current["temp_c"],
        "Hissedilen Sıcaklık (°C)": current["feelslike_c"],
        "Nem (%)": current["humidity"],
        "Rüzgar Hızı (km/s)": current["wind_kph"],
        "Veri Saati": location["localtime"],
        "season": convert_to_season(datetime.now().month)
    }

    print("\n📊 Hava Durumu Bilgileri:")
    for etiket, bilgi in hava_durumu.items():
        if etiket != "season":
            print(f"{etiket}: {bilgi}")

    return hava_durumu

def convert_to_season(month):
    if month in [12, 1, 2]: return "Winter"
    elif month in [3, 4, 5]: return "Spring"
    elif month in [6, 7, 8]: return "Summer"
    else: return "Fall"

# 2. Kıyafet verisini yükle ve hazırlık yap
def load_and_prepare_data():
    df = pd.read_csv("fashion_data_2018_2022.csv")
    df = df[["category", "season", "gender"]]
    df = df.dropna()

    # Cinsiyete göre mantıksız kategorileri ayıkla
    female_only_items = ["Dress", "Skirt"]
    male_only_items = ["Suit", "Blazer"]

    df = df[~((df["gender"] == "Male") & (df["category"].isin(female_only_items)))]
    df = df[~((df["gender"] == "Female") & (df["category"].isin(male_only_items)))]

    le_season = LabelEncoder()
    le_gender = LabelEncoder()
    le_category = LabelEncoder()

    df["season_enc"] = le_season.fit_transform(df["season"])
    df["gender_enc"] = le_gender.fit_transform(df["gender"])
    df["category_enc"] = le_category.fit_transform(df["category"])

    X = df[["season_enc", "gender_enc"]]
    y = df["category_enc"]

    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X, y)

    return model, le_season, le_gender, le_category

# 3. Tahmin yapan fonksiyon
def predict_outfit(model, season, gender, le_season, le_gender, le_category):
    input_data = pd.DataFrame.from_dict({
        "season_enc": [le_season.transform([season])[0]],
        "gender_enc": [le_gender.transform([gender])[0]]
    })
    prediction = model.predict(input_data)[0]
    return le_category.inverse_transform([prediction])[0]

# 4. Ana fonksiyon
def recommend(city, gender="Unisex"):
    weather = get_weather(city)
    model, le_season, le_gender, le_category = load_and_prepare_data()
    outfit = predict_outfit(
        model,
        season=weather["season"],
        gender=gender,
        le_season=le_season,
        le_gender=le_gender,
        le_category=le_category
    )
    print(f"👗 Önerilen Kıyafet: {outfit}")


print(recommend("Istanbul", gender="Male"))


📊 Hava Durumu Bilgileri:
Şehir: Istanbul
Ülke: Turkey
Durum: Açık
Sıcaklık (°C): 15.2
Hissedilen Sıcaklık (°C): 15.2
Nem (%): 94
Rüzgar Hızı (km/s): 6.5
Veri Saati: 2025-05-17 23:36
👗 Önerilen Kıyafet: Blouse
None


In [ ]:
# weather_outfit_ml.py

import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import random

# 1. Hava durumu verisini WeatherAPI'den al
API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=tr"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"🚨 Hava durumu alınamadı: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    hava_durumu = {
        "Şehir": location["name"],
        "Ülke": location["country"],
        "Durum": current["condition"]["text"],
        "Sıcaklık (°C)": current["temp_c"],
        "Hissedilen Sıcaklık (°C)": current["feelslike_c"],
        "Nem (%)": current["humidity"],
        "Rüzgar Hızı (km/s)": current["wind_kph"],
        "Veri Saati": location["localtime"],
        "season": convert_to_season(datetime.now().month)
    }

    print("\n📊 Hava Durumu Bilgileri:")
    for etiket, bilgi in hava_durumu.items():
        if etiket != "season":
            print(f"{etiket}: {bilgi}")

    return hava_durumu

def convert_to_season(month):
    if month in [12, 1, 2]: return "Winter"
    elif month in [3, 4, 5]: return "Spring"
    elif month in [6, 7, 8]: return "Summer"
    else: return "Fall"

# 2. Kıyafet verisini yükle ve hazırlık yap
def load_and_prepare_data():
    df = pd.read_csv("fashion_data_2018_2022.csv")
    df = df[["category", "season", "gender"]]
    df = df.dropna()

    # Cinsiyete göre mantıksız kategorileri ayıkla
    female_only_items = ["Dress", "Skirt"]
    male_only_items = ["Suit", "Blazer"]

    df = df[~((df["gender"] == "Male") & (df["category"].isin(female_only_items)))]
    df = df[~((df["gender"] == "Female") & (df["category"].isin(male_only_items)))]

    le_season = LabelEncoder()
    le_gender = LabelEncoder()
    le_category = LabelEncoder()

    df["season_enc"] = le_season.fit_transform(df["season"])
    df["gender_enc"] = le_gender.fit_transform(df["gender"])
    df["category_enc"] = le_category.fit_transform(df["category"])

    X = df[["season_enc", "gender_enc"]]
    y = df["category_enc"]

    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X, y)

    df["category_label"] = df["category"]
    return model, le_season, le_gender, le_category, df

# 3. Kombin öneren fonksiyon
def suggest_combination(model, season, gender, le_season, le_gender, le_category, df):
    input_data = pd.DataFrame.from_dict({
        "season_enc": [le_season.transform([season])[0]],
        "gender_enc": [le_gender.transform([gender])[0]]
    })
    prediction = model.predict(input_data)[0]
    main_item = le_category.inverse_transform([prediction])[0]

    # Kombin için aynı sezon ve cinsiyette rastgele 2 parça daha öner
    subset = df[(df["season"] == season) & (df["gender"] == gender)]
    additional_items = subset["category_label"].sample(n=2, replace=True, random_state=random.randint(0,1000)).tolist()
    combination = [main_item] + additional_items
    return combination

# 4. Ana fonksiyon
def recommend(city, gender="Unisex"):
    weather = get_weather(city)
    model, le_season, le_gender, le_category, df = load_and_prepare_data()
    combination = suggest_combination(
        model,
        season=weather["season"],
        gender=gender,
        le_season=le_season,
        le_gender=le_gender,
        le_category=le_category,
        df=df
    )
    print("👗 Önerilen Kombin:")
    for parca in combination:
        print("-", parca)

# Örnek çağrı:
print(recommend("Istanbul", gender="Male"))


📊 Hava Durumu Bilgileri:
Şehir: Istanbul
Ülke: Turkey
Durum: Açık
Sıcaklık (°C): 15.2
Hissedilen Sıcaklık (°C): 15.2
Nem (%): 94
Rüzgar Hızı (km/s): 6.5
Veri Saati: 2025-05-17 23:36
👗 Önerilen Kombin:
- Blouse
- Shoes
- Shorts
None


In [ ]:
# weather_outfit_ml.py

import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import random

# 1. WeatherAPI'den hava durumu verisi al
API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f" Weather data fetch failed: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    weather_info = {
        "City": location["name"],
        "Country": location["country"],
        "Condition": current["condition"]["text"],
        "Temperature (°C)": current["temp_c"],
        "Feels Like (°C)": current["feelslike_c"],
        "Humidity (%)": current["humidity"],
        "Wind Speed (km/h)": current["wind_kph"],
        "Local Time": location["localtime"],
        "season": convert_to_season(datetime.now().month)
    }

    print("\n Weather Information:")
    for label, value in weather_info.items():
        if label != "season":
            print(f"{label}: {value}")

    return weather_info

def convert_to_season(month):
    if month in [12, 1, 2]: return "Winter"
    elif month in [3, 4, 5]: return "Spring"
    elif month in [6, 7, 8]: return "Summer"
    else: return "Fall"

# 2. Load and prepare fashion data
def load_and_prepare_data():
    df = pd.read_csv("fashion_data_2018_2022.csv")
    df = df[["category", "season", "gender"]]
    df = df.dropna()

    # Remove illogical gender-category combinations
    female_only_items = ["Dress", "Skirt"]
    male_only_items = ["Suit", "Blazer"]

    df = df[~((df["gender"] == "Male") & (df["category"].isin(female_only_items)))]
    df = df[~((df["gender"] == "Female") & (df["category"].isin(male_only_items)))]

    le_season = LabelEncoder()
    le_gender = LabelEncoder()
    le_category = LabelEncoder()

    df["season_enc"] = le_season.fit_transform(df["season"])
    df["gender_enc"] = le_gender.fit_transform(df["gender"])
    df["category_enc"] = le_category.fit_transform(df["category"])

    X = df[["season_enc", "gender_enc"]]
    y = df["category_enc"]

    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X, y)

    df["category_label"] = df["category"]
    return model, le_season, le_gender, le_category, df

# 3. Suggest a clothing combination
def suggest_combination(model, season, gender, le_season, le_gender, le_category, df):
    input_data = pd.DataFrame.from_dict({
        "season_enc": [le_season.transform([season])[0]],
        "gender_enc": [le_gender.transform([gender])[0]]
    })
    prediction = model.predict(input_data)[0]
    main_item = le_category.inverse_transform([prediction])[0]

    # Randomly suggest 2 more compatible items
    subset = df[(df["season"] == season) & (df["gender"] == gender)]
    additional_items = subset["category_label"].dropna().sample(n=2, replace=True, random_state=random.randint(0,1000)).tolist()
    combination = [main_item] + additional_items
    combination = [item for item in combination if item is not None]
    return combination

# 4. Main function
def recommend(city, gender="Unisex"):
    weather = get_weather(city)
    model, le_season, le_gender, le_category, df = load_and_prepare_data()
    combination = suggest_combination(
        model,
        season=weather["season"],
        gender=gender,
        le_season=le_season,
        le_gender=le_gender,
        le_category=le_category,
        df=df
    )
    print(" Suggested Outfit:")
    for item in combination:
        print("-", item)

# Example call:
print(recommend("Istanbul", gender="Male"))


 Weather Information:
City: Istanbul
Country: Turkey
Condition: Clear
Temperature (°C): 15.2
Feels Like (°C): 15.2
Humidity (%): 94
Wind Speed (km/h): 6.5
Local Time: 2025-05-17 23:39
 Suggested Outfit:
- Blouse
- Shoes
- Shirt
None


In [ ]:
# weather_outfit_ml.py

import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import random

# 1. WeatherAPI'den hava durumu verisi al
API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Weather data fetch failed: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    weather_info = {
        "City": location["name"],
        "Country": location["country"],
        "Condition": current["condition"]["text"],
        "Temperature (°C)": current["temp_c"],
        "Feels Like (°C)": current["feelslike_c"],
        "Humidity (%)": current["humidity"],
        "Wind Speed (km/h)": current["wind_kph"],
        "Local Time": location["localtime"],
        "season": convert_to_season(datetime.now().month)
    }

    print("\nWeather Information:")
    for label, value in weather_info.items():
        if label != "season":
            print(f"{label}: {value}")

    return weather_info

def convert_to_season(month):
    if month in [12, 1, 2]: return "Winter"
    elif month in [3, 4, 5]: return "Spring"
    elif month in [6, 7, 8]: return "Summer"
    else: return "Fall"

# 2. Load and prepare fashion data
def load_and_prepare_data():
    df = pd.read_csv("fashion_data_2018_2022.csv")
    df = df[["category", "season", "gender"]]
    df = df.dropna()

    # Remove illogical gender-category combinations
    female_only_items = ["Dress", "Skirt"]
    male_only_items = ["Suit", "Blazer"]

    df = df[~((df["gender"] == "Male") & (df["category"].isin(female_only_items)))]
    df = df[~((df["gender"] == "Female") & (df["category"].isin(male_only_items)))]

    df = df[df["category"].notnull() & df["season"].notnull() & df["gender"].notnull()]

    le_season = LabelEncoder()
    le_gender = LabelEncoder()
    le_category = LabelEncoder()

    df["season_enc"] = le_season.fit_transform(df["season"])
    df["gender_enc"] = le_gender.fit_transform(df["gender"])
    df["category_enc"] = le_category.fit_transform(df["category"])

    X = df[["season_enc", "gender_enc"]]
    y = df["category_enc"]

    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X, y)

    df["category_label"] = df["category"]
    return model, le_season, le_gender, le_category, df

# 3. Suggest a clothing combination
def suggest_combination(model, season, gender, le_season, le_gender, le_category, df):
    input_data = pd.DataFrame.from_dict({
        "season_enc": [le_season.transform([season])[0]],
        "gender_enc": [le_gender.transform([gender])[0]]
    })
    prediction = model.predict(input_data)[0]
    main_item = le_category.inverse_transform([prediction])[0]

    # Randomly suggest 2 more compatible items
    subset = df[(df["season"] == season) & (df["gender"] == gender)]
    subset = subset[subset["category_label"].notnull()]
    additional_items = subset["category_label"].sample(n=2, replace=True, random_state=random.randint(0,1000)).tolist()
    combination = [main_item] + additional_items
    combination = [item for item in combination if isinstance(item, str) and item.strip().lower() != "none"]
    return combination

# 4. Main function
def recommend(city, gender="Unisex"):
    weather = get_weather(city)
    model, le_season, le_gender, le_category, df = load_and_prepare_data()
    combination = suggest_combination(
        model,
        season=weather["season"],
        gender=gender,
        le_season=le_season,
        le_gender=le_gender,
        le_category=le_category,
        df=df
    )
    print("Suggested Outfit:")
    for item in combination:
        print("-", item)

# Example call:
print(recommend("Istanbul", gender="Male"))



Weather Information:
City: Istanbul
Country: Turkey
Condition: Clear
Temperature (°C): 15.2
Feels Like (°C): 15.2
Humidity (%): 94
Wind Speed (km/h): 6.5
Local Time: 2025-05-17 23:39
Suggested Outfit:
- Blouse
- Shirt
- Shoes
None


In [ ]:
# weather_outfit_ml.py

import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import random

# 1. WeatherAPI'den hava durumu verisi al
API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Weather data fetch failed: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    weather_info = {
        "City": location["name"],
        "Country": location["country"],
        "Condition": current["condition"]["text"],
        "Temperature (°C)": current["temp_c"],
        "Feels Like (°C)": current["feelslike_c"],
        "Humidity (%)": current["humidity"],
        "Wind Speed (km/h)": current["wind_kph"],
        "Local Time": location["localtime"],
        "season": convert_to_season(datetime.now().month)
    }

    print("\nWeather Information:")
    for label, value in weather_info.items():
        if label != "season":
            print(f"{label}: {value}")

    return weather_info

def convert_to_season(month):
    if month in [12, 1, 2]: return "Winter"
    elif month in [3, 4, 5]: return "Spring"
    elif month in [6, 7, 8]: return "Summer"
    else: return "Fall"

# 2. Load and prepare fashion data
def load_and_prepare_data():
    df = pd.read_csv("fashion_data_2018_2022.csv")
    df = df[["category", "season", "gender"]]
    df = df.dropna()

    # Remove illogical gender-category combinations
    female_only_items = ["Dress", "Skirt"]
    male_only_items = ["Suit", "Blazer"]

    df = df[~((df["gender"].str.lower() == "male") & (df["category"].isin(female_only_items)))]
    df = df[~((df["gender"].str.lower() == "female") & (df["category"].isin(male_only_items)))]

    df = df[df["category"].notnull() & df["season"].notnull() & df["gender"].notnull()]

    df["gender"] = df["gender"].str.capitalize()
    le_season = LabelEncoder()
    le_gender = LabelEncoder()
    le_category = LabelEncoder()

    df["season_enc"] = le_season.fit_transform(df["season"])
    df["gender_enc"] = le_gender.fit_transform(df["gender"])
    df["category_enc"] = le_category.fit_transform(df["category"])

    X = df[["season_enc", "gender_enc"]]
    y = df["category_enc"]

    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X, y)

    df["category_label"] = df["category"]
    return model, le_season, le_gender, le_category, df

# 3. Suggest a clothing combination
def suggest_combination(model, season, gender, le_season, le_gender, le_category, df):
    gender = gender.capitalize()
    input_data = pd.DataFrame.from_dict({
        "season_enc": [le_season.transform([season])[0]],
        "gender_enc": [le_gender.transform([gender])[0]]
    })
    prediction = model.predict(input_data)[0]
    main_item = le_category.inverse_transform([prediction])[0]

    # Randomly suggest 2 more unique compatible items
    subset = df[(df["season"] == season) & (df["gender"] == gender)]
    subset = subset[subset["category_label"].notnull() & (subset["category_label"] != main_item)]
    additional_items = subset["category_label"].drop_duplicates().sample(n=min(2, len(subset["category_label"].drop_duplicates())), replace=False, random_state=random.randint(0,1000)).tolist()
    combination = [main_item] + additional_items
    combination = [item for item in combination if isinstance(item, str) and item.strip().lower() != "none"]
    return combination

# 4. Main function

def recommend():
    city = input("Enter city: ")
    gender = input("Enter gender (Male/Female/Unisex): ").capitalize()
    weather = get_weather(city)
    model, le_season, le_gender, le_category, df = load_and_prepare_data()
    combination = suggest_combination(
        model,
        season=weather["season"],
        gender=gender,
        le_season=le_season,
        le_gender=le_gender,
        le_category=le_category,
        df=df
    )
    print("Suggested Outfit:")
    for item in combination:
        print("-", item)

# Run the recommender interactively
recommend()


Enter city: ankara
Enter gender (Male/Female/Unisex): male

Weather Information:
City: Ankara
Country: Turkey
Condition: Clear
Temperature (°C): 19.2
Feels Like (°C): 19.2
Humidity (%): 49
Wind Speed (km/h): 8.3
Local Time: 2025-05-17 23:48
Suggested Outfit:
- Blouse
- Shirt
- Shorts


In [ ]:
# weather_outfit_ml.py

import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import random

API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

# 1. Weather API

def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Weather data fetch failed: {response.status_code}, {response.text}")
    data = response.json()
    current = data["current"]
    location = data["location"]
    return {
        "season": convert_to_season(datetime.now().month),
        "temp": current["temp_c"],
        "humidity": current["humidity"],
        "condition": current["condition"]["text"],
        "city": location["name"]
    }

def convert_to_season(month):
    return ("Winter" if month in [12, 1, 2] else
            "Spring" if month in [3, 4, 5] else
            "Summer" if month in [6, 7, 8] else
            "Fall")

# 2. Load and clean dataset

def load_data():
    df = pd.read_csv("styles.csv", on_bad_lines='skip')
    df = df[["gender", "masterCategory", "subCategory", "season"]].dropna()
    df = df[df["gender"].str.lower().isin(["male", "female", "unisex"])]
    df["gender"] = df["gender"].str.capitalize()
    df["season"] = df["season"].str.capitalize()
    df = df[df["season"].isin(["Summer", "Winter", "Spring", "Fall"])]
    return df

# 3. Train models for each masterCategory

def train_models(df):
    label_encoders = {}
    models = {}

    for category in df["masterCategory"].unique():
        subset = df[df["masterCategory"] == category].copy()
        le_gender = LabelEncoder()
        le_season = LabelEncoder()
        le_sub = LabelEncoder()

        subset["gender_enc"] = le_gender.fit_transform(subset["gender"])
        subset["season_enc"] = le_season.fit_transform(subset["season"])
        subset["sub_enc"] = le_sub.fit_transform(subset["subCategory"])

        X = subset[["gender_enc", "season_enc"]]
        y = subset["sub_enc"]

        model = KNeighborsClassifier(n_neighbors=3)
        model.fit(X, y)

        models[category] = model
        label_encoders[category] = {
            "gender": le_gender,
            "season": le_season,
            "subCategory": le_sub
        }

    return models, label_encoders

# 4. Predict category using model

def predict_category(models, label_encoders, category, gender, season):
    if category not in models:
        return "None"

    le = label_encoders[category]
    model = models[category]

    try:
        gender_enc = le["gender"].transform([gender])[0]
        season_enc = le["season"].transform([season])[0]
        X_pred = [[gender_enc, season_enc]]
        pred_enc = model.predict(X_pred)[0]
        return le["subCategory"].inverse_transform([pred_enc])[0]
    except:
        return "None"

# 5. Wrapper for UI

def get_outfit_by_selections(city, gender, include_top=True, include_bottom=True, include_footwear=True, include_accessory=True):
    weather = get_weather(city)
    df = load_data()
    gender = gender.capitalize()
    season = weather["season"]

    models, encoders = train_models(df)

    result = {}
    if include_top:
        result["Topwear"] = predict_category(models, encoders, "Apparel", gender, season)
    if include_bottom:
        result["Bottomwear"] = predict_category(models, encoders, "Bottomwear", gender, season)
    if include_footwear:
        result["Footwear"] = predict_category(models, encoders, "Footwear", gender, season)
    if include_accessory:
        result["Accessories"] = predict_category(models, encoders, "Accessories", gender, season)

    return weather, result


16ce454570964d39901204010250905

In [ ]:
# weather_outfit_ml.py

import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import random

API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

# 1. Weather API (English output)
def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch weather data: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    print("\nWeather Information:")
    print(f"City: {location['name']}")
    print(f"Country: {location['country']}")
    print(f"Condition: {current['condition']['text']}")
    print(f"Temperature (°C): {current['temp_c']}")
    print(f"Feels Like (°C): {current['feelslike_c']}")
    print(f"Humidity (%): {current['humidity']}")
    print(f"Wind Speed (km/h): {current['wind_kph']}")
    print(f"Local Time: {location['localtime']}")

    return {
        "season": convert_to_season(datetime.now().month),
        "temp": current["temp_c"],
        "humidity": current["humidity"],
        "condition": current["condition"]["text"],
        "city": location["name"]
    }

def convert_to_season(month):
    return (
        "Winter" if month in [12, 1, 2] else
        "Spring" if month in [3, 4, 5] else
        "Summer" if month in [6, 7, 8] else
        "Fall"
    )

# 2. Load and clean dataset
def load_data():
    df = pd.read_csv("styles.csv", on_bad_lines='skip')
    df = df[["gender", "masterCategory", "subCategory", "season"]].dropna()
    df = df[df["gender"].str.lower().isin(["male", "female", "unisex"])]
    df["gender"] = df["gender"].str.capitalize()
    df["season"] = df["season"].str.capitalize()
    df = df[df["season"].isin(["Summer", "Winter", "Spring", "Fall"])]
    return df

# 3. Train models for each masterCategory
def train_models(df):
    label_encoders = {}
    models = {}

    for category in df["masterCategory"].unique():
        subset = df[df["masterCategory"] == category].copy()
        if subset["subCategory"].nunique() < 2:
            continue

        le_gender = LabelEncoder()
        le_season = LabelEncoder()
        le_sub = LabelEncoder()

        subset["gender_enc"] = le_gender.fit_transform(subset["gender"])
        subset["season_enc"] = le_season.fit_transform(subset["season"])
        subset["sub_enc"] = le_sub.fit_transform(subset["subCategory"])

        X = subset[["gender_enc", "season_enc"]]
        y = subset["sub_enc"]

        model = KNeighborsClassifier(n_neighbors=3)
        model.fit(X, y)

        models[category] = model
        label_encoders[category] = {
            "gender": le_gender,
            "season": le_season,
            "subCategory": le_sub
        }

    return models, label_encoders

# 4. Predict category using model
def predict_category(models, label_encoders, category, gender, season):
    if category not in models:
        return "None"

    le = label_encoders[category]
    model = models[category]

    try:
        gender_enc = le["gender"].transform([gender])[0]
        season_enc = le["season"].transform([season])[0]
        X_pred = [[gender_enc, season_enc]]
        pred_enc = model.predict(X_pred)[0]
        return le["subCategory"].inverse_transform([pred_enc])[0]
    except:
        return "None"

# 5. Wrapper for UI
def get_outfit_by_selections(city, gender, include_top=True, include_bottom=True, include_footwear=True, include_accessory=True):
    weather = get_weather(city)
    df = load_data()
    gender = gender.capitalize()
    season = weather["season"]

    models, encoders = train_models(df)

    result = {}
    if include_top:
        result["Topwear"] = predict_category(models, encoders, "Apparel", gender, season)
    if include_bottom:
        result["Bottomwear"] = predict_category(models, encoders, "Bottomwear", gender, season)
    if include_footwear:
        result["Footwear"] = predict_category(models, encoders, "Footwear", gender, season)
    if include_accessory:
        result["Accessories"] = predict_category(models, encoders, "Accessories", gender, season)

    return weather, result

# 6. Run interactively
if __name__ == "__main__":
    city = input("Enter your city: ")
    gender = input("Enter your gender (Male/Female/Unisex): ")
    weather, outfit = get_outfit_by_selections(city, gender)

    print("\nSuggested Outfit:")
    for k, v in outfit.items():
        print(f"{k}: {v}")


Enter your city: eskisehir 
Enter your gender (Male/Female/Unisex): female

Weather Information:
City: Eskisehir
Country: Turkey
Condition: Partly cloudy
Temperature (°C): 15.3
Feels Like (°C): 15.3
Humidity (%): 82
Wind Speed (km/h): 9.0
Local Time: 2025-05-18 00:19

Suggested Outfit:
Topwear: None
Bottomwear: None
Footwear: None
Accessories: None


In [ ]:
import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

# 1. Hava durumu alma
def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch weather data: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    print("\nWeather Information:")
    print(f"City: {location['name']}")
    print(f"Country: {location['country']}")
    print(f"Condition: {current['condition']['text']}")
    print(f"Temperature (°C): {current['temp_c']}")
    print(f"Feels Like (°C): {current['feelslike_c']}")
    print(f"Humidity (%): {current['humidity']}")
    print(f"Wind Speed (km/h): {current['wind_kph']}")
    print(f"Local Time: {location['localtime']}")

    return {
        "season": convert_to_season(datetime.now().month),
        "temp": current["temp_c"],
        "humidity": current["humidity"],
        "condition": current["condition"]["text"],
        "city": location["name"]
    }

def convert_to_season(month):
    return "Winter" if month in [12, 1, 2] else "Spring" if month in [3, 4, 5] else "Summer" if month in [6, 7, 8] else "Fall"

# 2. Veri setini yükle ve temizle
def load_data():
    df = pd.read_csv("styles.csv", on_bad_lines='skip')
    df = df[["gender", "subCategory", "season"]].dropna()

    # Normalize string değerleri
    df["gender"] = df["gender"].str.strip().str.capitalize()
    df["season"] = df["season"].str.strip().str.capitalize()
    df["subCategory"] = df["subCategory"].str.strip().str.title()

    df = df[df["gender"].isin(["Male", "Female", "Unisex"])]
    df = df[df["season"].isin(["Summer", "Winter", "Spring", "Fall"])]

    # Grup etiketleri (bu listeyi daha sonra genişletebiliriz)
    topwear = ["Tshirts", "Shirts", "Sweatshirts", "Jackets", "Sweaters", "Tops"]
    bottomwear = ["Jeans", "Trousers", "Shorts", "Track Pants", "Skirts", "Leggings"]
    footwear = ["Sports Shoes", "Casual Shoes", "Sandals", "Flip Flops", "Sneakers", "Boots", "Heels"]

    df["category"] = df["subCategory"].apply(
        lambda x: "Topwear" if x in topwear else
                  "Bottomwear" if x in bottomwear else
                  "Footwear" if x in footwear else None
    )

    df = df.dropna(subset=["category"])

    if df.empty:
        raise ValueError("Veri kümesi boş. Uygun kıyafet bulunamadı. Lütfen kategorileri kontrol edin.")
    return df

# 3. Her kategori için model eğit
def train_models(df):
    models = {}
    encoders = {}

    for cat in ["Topwear", "Bottomwear", "Footwear"]:
        subset = df[df["category"] == cat]
        if subset.empty:
            print(f"Uyarı: {cat} için yeterli veri yok, atlanıyor.")
            continue

        le_gender = LabelEncoder()
        le_season = LabelEncoder()
        le_sub = LabelEncoder()

        subset["gender_enc"] = le_gender.fit_transform(subset["gender"])
        subset["season_enc"] = le_season.fit_transform(subset["season"])
        subset["sub_enc"] = le_sub.fit_transform(subset["subCategory"])

        X = subset[["gender_enc", "season_enc"]]
        y = subset["sub_enc"]

        model = KNeighborsClassifier(n_neighbors=3)
        model.fit(X, y)

        models[cat] = model
        encoders[cat] = (le_gender, le_season, le_sub)

    return models, encoders

# 4. Her kategori için tahmin yap
def predict(models, encoders, gender, season):
    outfit = {}
    for cat in models:
        le_gender, le_season, le_sub = encoders[cat]
        try:
            X_pred = [[le_gender.transform([gender])[0], le_season.transform([season])[0]]]
            pred_enc = models[cat].predict(X_pred)[0]
            outfit[cat] = le_sub.inverse_transform([pred_enc])[0]
        except:
            outfit[cat] = "No suggestion"
    return outfit

# 5. Ana fonksiyon
def get_outfit(city, gender):
    weather = get_weather(city)
    df = load_data()
    models, encoders = train_models(df)
    outfit = predict(models, encoders, gender.capitalize(), weather["season"])
    return weather, outfit

# 6. Komut satırından çalıştırma
if __name__ == "__main__":
    city = input("Enter your city: ")
    gender = input("Enter your gender (Male/Female/Unisex): ")
    weather, outfit = get_outfit(city, gender)

    print("\nSuggested Outfit:")
    print(f"- Topwear:    {outfit.get('Topwear', 'No suggestion')}")
    print(f"- Bottomwear: {outfit.get('Bottomwear', 'No suggestion')}")
    print(f"- Footwear:   {outfit.get('Footwear', 'No suggestion')}")


Enter your city: ANKARA
Enter your gender (Male/Female/Unisex): male

Weather Information:
City: Ankara
Country: Turkey
Condition: Clear
Temperature (°C): 19.0
Feels Like (°C): 19.0
Humidity (%): 52
Wind Speed (km/h): 8.6
Local Time: 2025-05-18 01:07
Uyarı: Topwear için yeterli veri yok, atlanıyor.
Uyarı: Bottomwear için yeterli veri yok, atlanıyor.

Suggested Outfit:
- Topwear:    No suggestion
- Bottomwear: No suggestion
- Footwear:   No suggestion


In [ ]:
# weather_outfit_ml.py
"""
Weather-aware Outfit Recommendation
===================================
Generates a full outfit (topwear + bottomwear or dress, shoes, accessory)
by combining live weather data with a machine-learning model trained on a
synthetic clothing dataset.
"""

import random
from datetime import datetime
from typing import Dict, Tuple

import pandas as pd
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder

API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

# 1. Weather API
def convert_to_season(month: int) -> str:
    return (
        "Winter" if month in [12, 1, 2] else
        "Spring" if month in [3, 4, 5] else
        "Summer" if month in [6, 7, 8] else
        "Fall"
    )

def get_weather(city: str) -> Dict[str, str]:
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    resp = requests.get(url, timeout=10)
    if resp.status_code != 200:
        raise RuntimeError(f"Weather API error – {resp.status_code}: {resp.text}")

    data = resp.json()
    cur, loc = data["current"], data["location"]

    print("\nWeather Information:")
    print(f"City: {loc['name']}")
    print(f"Country: {loc['country']}")
    print(f"Condition: {cur['condition']['text']}")
    print(f"Temperature (°C): {cur['temp_c']}")
    print(f"Feels Like (°C): {cur['feelslike_c']}")
    print(f"Humidity (%): {cur['humidity']}")
    print(f"Wind Speed (km/h): {cur['wind_kph']}")
    print(f"Local Time: {loc['localtime']}\n")

    return {
        "season": convert_to_season(datetime.now().month),
        "temp": cur["temp_c"],
        "humidity": cur["humidity"],
        "condition": cur["condition"]["text"],
        "city": loc["name"],
    }

# 2. Synthetic dataset generation
GENDERS = ["Male", "Female", "Unisex"]
SEASONS = ["Spring", "Summer", "Fall", "Winter"]

TOPWEAR = {
    "Male":    ["T-shirt", "Shirt", "Polo", "Sweater", "Jacket", "Sweatshirt"],
    "Female":  ["Blouse", "Crop-top", "Sweater", "Cardigan", "Jacket", "T-shirt"],
    "Unisex":  ["Hoodie", "Sweatshirt", "Denim Jacket", "Raincoat", "T-shirt"],
}

BOTTOMWEAR = {
    "Male":    ["Jeans", "Chinos", "Shorts", "Joggers", "Cargo Pants"],
    "Female":  ["Jeans", "Skirt", "Shorts", "Trousers"],
    "Unisex":  ["Jeans", "Cargo Pants", "Shorts", "Joggers"],
}

DRESSES = {
    "Female": [
        "Summer Dress", "Evening Dress", "Sweater Dress", "Cocktail Dress",
    ]
}

SHOES = ["Sneakers", "Loafers", "Boots", "Sandals", "Heels", "Running Shoes"]
ACCESSORIES = [
    "Cap", "Scarf", "Watch", "Sunglasses", "Backpack", "Belt", "Handbag",
]

def make_outfit(n_rows: int = 5000, *, seed: int = 42) -> pd.DataFrame:
    rnd = random.Random(seed)
    rows = []
    for _ in range(n_rows):
        gender = rnd.choice(GENDERS)
        season = rnd.choice(SEASONS)
        use_dress = gender == "Female" and rnd.random() < 0.30

        if use_dress:
            dress = rnd.choice(DRESSES["Female"])
            top, bottom = None, None
        else:
            dress = None
            top = rnd.choice(TOPWEAR[gender])
            bottom = rnd.choice(BOTTOMWEAR[gender])

        shoe = rnd.choice(SHOES)
        acc = rnd.choice(ACCESSORIES)

        rows.append({
            "gender": gender,
            "season": season,
            "topwear": top,
            "bottomwear": bottom,
            "dress": dress,
            "shoes": shoe,
            "accessory": acc,
        })
    return pd.DataFrame(rows)

# 3. ML training and prediction

def _encode(df: pd.DataFrame):
    encoders = {}
    df_enc = df.copy()
    for col in df.columns:
        le = LabelEncoder()
        df_enc[col] = le.fit_transform(df[col].fillna("None"))
        encoders[col] = le
    return df_enc, encoders

def train_model(df: pd.DataFrame):
    y_cols = ["topwear", "bottomwear", "dress", "shoes", "accessory"]
    df_enc, encs = _encode(df[["gender", "season", *y_cols]])
    X = df_enc[["gender", "season"]]
    Y = df_enc[y_cols]
    rf = RandomForestClassifier(n_estimators=200, random_state=42)
    model = MultiOutputClassifier(rf)
    model.fit(X, Y)
    return model, encs

def predict_outfit(model, encs, gender: str, season: str):
    gender = gender.capitalize()
    season = season.capitalize()
    X_pred = [[
        encs["gender"].transform([gender])[0],
        encs["season"].transform([season])[0],
    ]]
    preds = model.predict(X_pred)[0]
    y_cols = ["topwear", "bottomwear", "dress", "shoes", "accessory"]
    decoded = {
        col: encs[col].inverse_transform([pred])[0] for col, pred in zip(y_cols, preds)
    }
    decoded = {k: (None if v == "None" else v) for k, v in decoded.items()}
    if decoded["dress"] is not None:
        decoded["topwear"] = decoded["bottomwear"] = None
    return decoded

# 4. Interface

def get_recommendation(city: str, gender: str) -> Tuple[Dict, Dict]:
    weather = get_weather(city)
    season = weather["season"]
    global _MODEL, _ENCS
    if "_MODEL" not in globals():
        df = make_outfit()
        _MODEL, _ENCS = train_model(df)
    outfit = predict_outfit(_MODEL, _ENCS, gender, season)
    return weather, outfit

if __name__ == "__main__":
    city = input("Enter your city: ")
    gender = input("Enter your gender (Male/Female/Unisex): ")
    weather_info, outfit = get_recommendation(city, gender)

    print("Suggested Outfit:")
    if outfit["dress"]:
        print(f"- {outfit['dress']}")
    else:
        print(f"- Topwear:    {outfit['topwear']}")
        print(f"- Bottomwear: {outfit['bottomwear']}")
    print(f"- Shoes:      {outfit['shoes']}")
    print(f"- Accessory:  {outfit['accessory']}")

Enter your city: ankara
Enter your gender (Male/Female/Unisex): male

Weather Information:
City: Ankara
Country: Turkey
Condition: Clear
Temperature (°C): 19.2
Feels Like (°C): 19.2
Humidity (%): 52
Wind Speed (km/h): 8.6
Local Time: 2025-05-18 00:43

Suggested Outfit:
- Topwear:    Shirt
- Bottomwear: Chinos
- Shoes:      Running Shoes
- Accessory:  Backpack


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# weather_outfit_ml.py
"""
Weather-aware Outfit Recommendation
===================================
Generates a full outfit (topwear + bottomwear or dress, shoes, accessory)
by combining live weather data with a machine-learning model trained on a
synthetic clothing dataset.
"""

import random
from datetime import datetime
from typing import Dict, Tuple

import pandas as pd
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

# 1. Weather API
def convert_to_season(month: int) -> str:
    return (
        "Winter" if month in [12, 1, 2] else
        "Spring" if month in [3, 4, 5] else
        "Summer" if month in [6, 7, 8] else
        "Fall"
    )

def get_weather(city: str) -> Dict[str, str]:
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    resp = requests.get(url, timeout=10)
    if resp.status_code != 200:
        raise RuntimeError(f"Weather API error – {resp.status_code}: {resp.text}")

    data = resp.json()
    cur, loc = data["current"], data["location"]

    print("\nWeather Information:")
    print(f"City: {loc['name']}")
    print(f"Country: {loc['country']}")
    print(f"Condition: {cur['condition']['text']}")
    print(f"Temperature (°C): {cur['temp_c']}")
    print(f"Feels Like (°C): {cur['feelslike_c']}")
    print(f"Humidity (%): {cur['humidity']}")
    print(f"Wind Speed (km/h): {cur['wind_kph']}")
    print(f"Local Time: {loc['localtime']}\n")

    return {
        "season": convert_to_season(datetime.now().month),
        "temp": cur["temp_c"],
        "humidity": cur["humidity"],
        "condition": cur["condition"]["text"],
        "city": loc["name"],
    }

# 2. Synthetic dataset generation
GENDERS = ["Male", "Female", "Unisex"]
SEASONS = ["Spring", "Summer", "Fall", "Winter"]

TOPWEAR = {
    "Male":    ["T-shirt", "Shirt", "Polo", "Sweater", "Jacket", "Sweatshirt"],
    "Female":  ["Blouse", "Crop-top", "Sweater", "Cardigan", "Jacket", "T-shirt"],
    "Unisex":  ["Hoodie", "Sweatshirt", "Denim Jacket", "Raincoat", "T-shirt"],
}

BOTTOMWEAR = {
    "Male":    ["Jeans", "Chinos", "Shorts", "Joggers", "Cargo Pants"],
    "Female":  ["Jeans", "Skirt", "Shorts", "Trousers"],
    "Unisex":  ["Jeans", "Cargo Pants", "Shorts", "Joggers"],
}

DRESSES = {
    "Female": [
        "Summer Dress", "Evening Dress", "Sweater Dress", "Cocktail Dress",
    ]
}

SHOES = ["Sneakers", "Loafers", "Boots", "Sandals", "Heels", "Running Shoes"]
ACCESSORIES = [
    "Cap", "Scarf", "Watch", "Sunglasses", "Backpack", "Belt", "Handbag",
]

def make_outfit(n_rows: int = 5000, *, seed: int = 42) -> pd.DataFrame:
    rnd = random.Random(seed)
    rows = []
    for _ in range(n_rows):
        gender = rnd.choice(GENDERS)
        season = rnd.choice(SEASONS)
        use_dress = gender == "Female" and rnd.random() < 0.30

        if use_dress:
            dress = rnd.choice(DRESSES["Female"])
            top, bottom = "-", "-"
        else:
            dress = "-"
            top = rnd.choice(TOPWEAR[gender])
            bottom = rnd.choice(BOTTOMWEAR[gender])

        shoe = rnd.choice(SHOES)
        acc = rnd.choice(ACCESSORIES)

        rows.append({
            "gender": gender,
            "season": season,
            "topwear": top,
            "bottomwear": bottom,
            "dress": dress,
            "shoes": shoe,
            "accessory": acc,
        })
    return pd.DataFrame(rows)

# 3. ML training and prediction

def _encode(df: pd.DataFrame):
    encoders = {}
    df_enc = df.copy()
    for col in df.columns:
        le = LabelEncoder()
        df_enc[col] = le.fit_transform(df[col])
        encoders[col] = le
    return df_enc, encoders

def train_model(df: pd.DataFrame):
    y_cols = ["topwear", "bottomwear", "dress", "shoes", "accessory"]
    df_enc, encs = _encode(df[["gender", "season", *y_cols]])
    X = df_enc[["gender", "season"]].to_numpy()
    Y = df_enc[y_cols]
    rf = RandomForestClassifier(n_estimators=200, random_state=42)
    model = MultiOutputClassifier(rf)
    model.fit(X, Y)
    return model, encs

def predict_outfit(model, encs, gender: str, season: str):
    gender = gender.capitalize()
    season = season.capitalize()
    X_pred = np.array([[
        encs["gender"].transform([gender])[0],
        encs["season"].transform([season])[0]
    ]])
    preds = model.predict(X_pred)[0]
    y_cols = ["topwear", "bottomwear", "dress", "shoes", "accessory"]
    decoded = {
        col: encs[col].inverse_transform([pred])[0] for col, pred in zip(y_cols, preds)
    }
    return decoded

# 4. Interface

def get_recommendation(city: str, gender: str) -> Tuple[Dict, Dict]:
    weather = get_weather(city)
    season = weather["season"]
    global _MODEL, _ENCS
    if "_MODEL" not in globals():
        df = make_outfit()
        _MODEL, _ENCS = train_model(df)
    outfit = predict_outfit(_MODEL, _ENCS, gender, season)
    return weather, outfit

if __name__ == "__main__":
    city = input("Enter your city: ")
    gender = input("Enter your gender (Male/Female/Unisex): ")
    weather_info, outfit = get_recommendation(city, gender)

    print("Suggested Outfit:")
    if outfit["dress"] != "-":
        print(f"- {outfit['dress']}")
    else:
        print(f"- Topwear:    {outfit['topwear']}")
        print(f"- Bottomwear: {outfit['bottomwear']}")
    print(f"- Shoes:      {outfit['shoes']}")
    print(f"- Accessory:  {outfit['accessory']}")


Enter your city: izmir
Enter your gender (Male/Female/Unisex): female

Weather Information:
City: Izmir
Country: Turkey
Condition: Patchy rain nearby
Temperature (°C): 16.4
Feels Like (°C): 16.4
Humidity (%): 88
Wind Speed (km/h): 7.9
Local Time: 2025-05-18 00:49

Suggested Outfit:
- None
- Shoes:      Running Shoes
- Accessory:  Belt


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

# 1. Hava durumu alma
def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch weather data: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    print("\nWeather Information:")
    print(f"City: {location['name']}")
    print(f"Country: {location['country']}")
    print(f"Condition: {current['condition']['text']}")
    print(f"Temperature (°C): {current['temp_c']}")
    print(f"Feels Like (°C): {current['feelslike_c']}")
    print(f"Humidity (%): {current['humidity']}")
    print(f"Wind Speed (km/h): {current['wind_kph']}")
    print(f"Local Time: {location['localtime']}")

    return {
        "season": convert_to_season(datetime.now().month),
        "temp": current["temp_c"],
        "humidity": current["humidity"],
        "condition": current["condition"]["text"],
        "city": location["name"]
    }

def convert_to_season(month):
    return "Winter" if month in [12, 1, 2] else "Spring" if month in [3, 4, 5] else "Summer" if month in [6, 7, 8] else "Fall"

# 2. Veri setini yükle ve temizle
def load_data():
    df = pd.read_csv("styles.csv", on_bad_lines='skip')
    df = df[["gender", "subCategory", "season"]].dropna()

    # Normalize string değerleri
    df["gender"] = df["gender"].str.strip().str.capitalize()
    df["season"] = df["season"].str.strip().str.capitalize()
    df["subCategory"] = df["subCategory"].str.strip().str.title()

    df = df[df["gender"].isin(["Male", "Female", "Unisex"])]
    df = df[df["season"].isin(["Summer", "Winter", "Spring", "Fall"])]

    # Grup etiketleri (bu listeyi daha sonra genişletebiliriz)
    topwear = ["Tshirts", "Shirts", "Sweatshirts", "Jackets", "Sweaters", "Tops"]
    bottomwear = ["Jeans", "Trousers", "Shorts", "Track Pants", "Skirts", "Leggings"]
    footwear = ["Sports Shoes", "Casual Shoes", "Sandals", "Flip Flops", "Sneakers", "Boots", "Heels"]

    df["category"] = df["subCategory"].apply(
        lambda x: "Topwear" if x in topwear else
                  "Bottomwear" if x in bottomwear else
                  "Footwear" if x in footwear else None
    )

    df = df.dropna(subset=["category"])

    if df.empty:
        raise ValueError("Veri kümesi boş. Uygun kıyafet bulunamadı. Lütfen kategorileri kontrol edin.")
    return df

# 3. Her kategori için model eğit
def train_models(df):
    models = {}
    encoders = {}

    for cat in ["Topwear", "Bottomwear", "Footwear"]:
        subset = df[df["category"] == cat]
        if subset.empty:
            print(f"Uyarı: {cat} için yeterli veri yok, atlanıyor.")
            continue

        le_gender = LabelEncoder()
        le_season = LabelEncoder()
        le_sub = LabelEncoder()

        subset["gender_enc"] = le_gender.fit_transform(subset["gender"])
        subset["season_enc"] = le_season.fit_transform(subset["season"])
        subset["sub_enc"] = le_sub.fit_transform(subset["subCategory"])

        X = subset[["gender_enc", "season_enc"]]
        y = subset["sub_enc"]

        model = KNeighborsClassifier(n_neighbors=3)
        model.fit(X, y)

        models[cat] = model
        encoders[cat] = (le_gender, le_season, le_sub)

    return models, encoders

# 4. Her kategori için tahmin yap
def predict(models, encoders, gender, season):
    outfit = {}
    for cat in models:
        le_gender, le_season, le_sub = encoders[cat]
        try:
            X_pred = [[le_gender.transform([gender])[0], le_season.transform([season])[0]]]
            pred_enc = models[cat].predict(X_pred)[0]
            outfit[cat] = le_sub.inverse_transform([pred_enc])[0]
        except:
            outfit[cat] = "No suggestion"
    return outfit

# 5. Ana fonksiyon
def get_outfit(city, gender):
    weather = get_weather(city)
    df = load_data()
    models, encoders = train_models(df)
    outfit = predict(models, encoders, gender.capitalize(), weather["season"])
    return weather, outfit

# 6. Komut satırından çalıştırma
if __name__ == "__main__":
    city = input("Enter your city: ")
    gender = input("Enter your gender (Male/Female/Unisex): ")
    weather, outfit = get_outfit(city, gender)

    print("\nSuggested Outfit:")
    print(f"- Topwear:    {outfit.get('Topwear', 'No suggestion')}")
    print(f"- Bottomwear: {outfit.get('Bottomwear', 'No suggestion')}")
    print(f"- Footwear:   {outfit.get('Footwear', 'No suggestion')}")


Enter your city: ankara
Enter your gender (Male/Female/Unisex): male

Weather Information:
City: Ankara
Country: Turkey
Condition: Clear
Temperature (°C): 19.0
Feels Like (°C): 19.0
Humidity (%): 52
Wind Speed (km/h): 8.6
Local Time: 2025-05-18 01:10
Uyarı: Topwear için yeterli veri yok, atlanıyor.
Uyarı: Bottomwear için yeterli veri yok, atlanıyor.

Suggested Outfit:
- Topwear:    No suggestion
- Bottomwear: No suggestion
- Footwear:   No suggestion


In [ ]:
import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch weather data: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    print("\nWeather Information:")
    print(f"City: {location['name']}")
    print(f"Country: {location['country']}")
    print(f"Condition: {current['condition']['text']}")
    print(f"Temperature (°C): {current['temp_c']}")
    print(f"Feels Like (°C): {current['feelslike_c']}")
    print(f"Humidity (%): {current['humidity']}")
    print(f"Wind Speed (km/h): {current['wind_kph']}")
    print(f"Local Time: {location['localtime']}")

    return {
        "season": convert_to_season(datetime.now().month),
        "temp": current["temp_c"],
        "humidity": current["humidity"],
        "condition": current["condition"]["text"],
        "city": location["name"]
    }

def convert_to_season(month):
    return "Winter" if month in [12, 1, 2] else "Spring" if month in [3, 4, 5] else "Summer" if month in [6, 7, 8] else "Fall"

def load_data():
    df = pd.read_csv("styles.csv", on_bad_lines='skip')
    df = df[["gender", "subCategory", "season"]].dropna().copy()

    df["gender"] = df["gender"].str.strip().str.lower()
    df["season"] = df["season"].str.strip().str.capitalize()
    df["subCategory"] = df["subCategory"].str.strip().str.title()

    df = df[df["gender"].isin(["male", "female", "unisex"])]
    df = df[df["season"].isin(["Summer", "Winter", "Spring", "Fall"])]

    topwear = ["Topwear", "Tshirts", "Shirts", "Sweatshirts", "Jackets", "Sweaters", "Tops"]
    bottomwear = ["Bottomwear", "Jeans", "Trousers", "Shorts", "Track Pants", "Skirts", "Leggings"]
    footwear = ["Shoes", "Sandal", "Flip Flops", "Sneakers", "Boots", "Heels"]

    df["category"] = df["subCategory"].apply(
        lambda x: "Topwear" if x in topwear else
                  "Bottomwear" if x in bottomwear else
                  "Footwear" if x in footwear else None
    )

    df = df.dropna(subset=["category"]).copy()

    if df.empty:
        raise ValueError("Veri kümesi boş. Uygun kıyafet bulunamadı.")
    return df

def train_models(df):
    models = {}
    encoders = {}

    for cat in ["Topwear", "Bottomwear", "Footwear"]:
        subset = df[df["category"] == cat].copy()
        if subset.empty:
            print(f"Uyarı: {cat} için yeterli veri yok.")
            continue

        le_gender = LabelEncoder()
        le_season = LabelEncoder()
        le_sub = LabelEncoder()

        subset["gender_enc"] = le_gender.fit_transform(subset["gender"])
        subset["season_enc"] = le_season.fit_transform(subset["season"])
        subset["sub_enc"] = le_sub.fit_transform(subset["subCategory"])

        X = subset[["gender_enc", "season_enc"]]
        y = subset["sub_enc"]

        model = KNeighborsClassifier(n_neighbors=3)
        model.fit(X, y)

        models[cat] = model
        encoders[cat] = (le_gender, le_season, le_sub)

    return models, encoders

def predict(models, encoders, gender, season):
    gender = gender.lower()
    outfit = {}
    for cat in models:
        le_gender, le_season, le_sub = encoders[cat]
        try:
            gender_enc = le_gender.transform([gender])[0]
            season_enc = le_season.transform([season])[0]
            pred_enc = models[cat].predict([[gender_enc, season_enc]])[0]
            outfit[cat] = le_sub.inverse_transform([pred_enc])[0]
        except Exception as e:
            outfit[cat] = "No suggestion"
    return outfit

def get_outfit(city, gender):
    weather = get_weather(city)
    df = load_data()
    models, encoders = train_models(df)
    outfit = predict(models, encoders, gender, weather["season"])
    return weather, outfit

if __name__ == "__main__":
    city = input("Enter your city: ")
    gender = input("Enter your gender (Male/Female/Unisex): ")
    weather, outfit = get_outfit(city, gender)

    print("\nSuggested Outfit:")
    print(f"- Topwear:    {outfit.get('Topwear', 'No suggestion')}")
    print(f"- Bottomwear: {outfit.get('Bottomwear', 'No suggestion')}")
    print(f"- Footwear:   {outfit.get('Footwear', 'No suggestion')}")


Enter your city: ESKİSEHİR
Enter your gender (Male/Female/Unisex): MALE

Weather Information:
City: Eskisehir
Country: Turkey
Condition: Light rain
Temperature (°C): 14.1
Feels Like (°C): 13.7
Humidity (%): 94
Wind Speed (km/h): 9.0
Local Time: 2025-05-18 01:18

Suggested Outfit:
- Topwear:    No suggestion
- Bottomwear: No suggestion
- Footwear:   No suggestion


In [ ]:
import pandas as pd
df = pd.read_csv("styles.csv", on_bad_lines='skip')
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44424 entries, 0 to 44423
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  44424 non-null  int64  
 1   gender              44424 non-null  object 
 2   masterCategory      44424 non-null  object 
 3   subCategory         44424 non-null  object 
 4   articleType         44424 non-null  object 
 5   baseColour          44409 non-null  object 
 6   season              44403 non-null  object 
 7   year                44423 non-null  float64
 8   usage               44107 non-null  object 
 9   productDisplayName  44417 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.4+ MB


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch


In [ ]:
df = df.dropna(subset=['season'])
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 44403 entries, 0 to 44423
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  44403 non-null  int64  
 1   gender              44403 non-null  object 
 2   masterCategory      44403 non-null  object 
 3   subCategory         44403 non-null  object 
 4   articleType         44403 non-null  object 
 5   baseColour          44388 non-null  object 
 6   season              44403 non-null  object 
 7   year                44403 non-null  float64
 8   usage               44086 non-null  object 
 9   productDisplayName  44396 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.7+ MB
None


In [ ]:
if 'usage' in df.columns:
  print("\n'usage' column values:")
  print(df['usage'].unique())
else:
  print("\n'usage' column not found in the DataFrame.")


'usage' column values:
['Casual' 'Ethnic' 'Formal' 'Sports' nan 'Smart Casual' 'Travel' 'Party'
 'Home']


In [ ]:
df = pd.read_csv("styles_düzenlenmiş.csv", sep=';')
print(df.columns)


Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDisplayName',
       'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')


In [ ]:
import pandas as pd

df = pd.read_csv("styles_düzenlenmiş.csv", sep=';', on_bad_lines='skip')

print(df['masterCategory'].unique())
print(df['subCategory'].unique())

FileNotFoundError: [Errno 2] No such file or directory: 'styles_düzenlenmiş.csv'

In [ ]:
import pandas as pd
df=pd.read_csv("styles_temizlenmis_kategoriler.csv")
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,Unnamed: 10,Unnamed: 11
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,NaN,NaN
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,NaN,NaN
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,NaN,NaN
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,NaN,NaN
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,NaN,NaN


In [6]:
import pandas as pd
df = pd.read_csv("styles_düzenlenmiş_kategoriler.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'styles_düzenlenmiş_kategoriler.csv'

In [1]:
import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Weather data could not be fetched: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    month = datetime.strptime(location['localtime'], "%Y-%m-%d %H:%M").month
    season = convert_to_season(month)

    return {
        "season": season,
        "temp": current["temp_c"],
        "feels_like": current["feelslike_c"],
        "humidity": current["humidity"],
        "wind_speed": current["wind_kph"],
        "condition": current["condition"]["text"].lower()
    }

def convert_to_season(month):
    if month in [12, 1, 2]: return "Winter"
    elif month in [3, 4, 5]: return "Spring"
    elif month in [6, 7, 8]: return "Summer"
    else: return "Fall"
def load_and_preprocess_data():
    df = pd.read_csv("styles_temizlenmis_kategoriler.csv", on_bad_lines='skip')
    df = df.drop(columns=[col for col in df.columns if col.startswith("Unnamed")])
    df = df[["id", "gender", "masterCategory", "subCategory", "articleType",
             "baseColour", "season", "usage", "productDisplayName"]].dropna().copy()

    df["gender"] = df["gender"].str.strip().str.lower()
    df["season"] = df["season"].str.strip().str.capitalize()
    df["subCategory"] = df["subCategory"].astype(str).str.strip().str.title()
    df["masterCategory"] = df["masterCategory"].astype(str).str.strip().str.title()
    df["articleType"] = df["articleType"].str.strip().str.title()
    df["usage"] = df["usage"].str.strip().str.title()

    valid_master = ["Apparel", "Accessories", "Footwear", "Personal Care", "Free Items", "Sporting Goods", "Home"]
    mask_invalid_master = ~df["masterCategory"].isin(valid_master) & df["subCategory"].isin(valid_master)
    df.loc[mask_invalid_master, ["masterCategory", "subCategory"]] = df.loc[mask_invalid_master, ["subCategory", "masterCategory"]].values

    df = df[df["gender"].isin(["men", "women", "unisex"])]
    df = df[df["season"].isin(["Summer", "Winter", "Spring", "Fall"])]

    def map_article_to_category(x):
        x = x.lower()
        if "shoe" in x or "sandal" in x or "flip flop" in x:
            return "Footwear"
        elif "shirt" in x or "tshirt" in x or "top" in x or "sweater" in x or "jacket" in x:
            return "Topwear"
        elif "jean" in x or "pant" in x or "trouser" in x or "short" in x or "skirt" in x or "legging" in x:
            return "Bottomwear"
        else:
            return None

    df["category"] = df["articleType"].apply(map_article_to_category)
    df = df.dropna(subset=["category"]).copy()

    def assign_weather_features(row):
        if row["season"] == "Winter": return {"temp_min": -5, "temp_max": 10}
        elif row["season"] == "Summer": return {"temp_min": 20, "temp_max": 40}
        else: return {"temp_min": 10, "temp_max": 25}

    weather_features = df.apply(assign_weather_features, axis=1, result_type='expand')
    df = pd.concat([df, weather_features], axis=1)
    return df

def train_rf_models(df):
    models, encoders = {}, {}
    for category in df["category"].unique():
        category_df = df[df["category"] == category].copy()
        le_gender, le_season, le_usage, le_article = LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()
        category_df["gender_enc"] = le_gender.fit_transform(category_df["gender"])
        category_df["season_enc"] = le_season.fit_transform(category_df["season"])
        category_df["usage_enc"] = le_usage.fit_transform(category_df["usage"])
        category_df["article_enc"] = le_article.fit_transform(category_df["articleType"])
        X = category_df[["gender_enc", "season_enc", "usage_enc", "temp_min", "temp_max"]]
        y = category_df["article_enc"]
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X, y)
        models[category] = model
        encoders[category] = (le_gender, le_season, le_usage, le_article)
    return models, encoders

def predict_outfit(models, encoders, df, weather, gender, usage):
    gender = gender.lower()
    usage = usage.title()
    outfit = {}
    for category, model in models.items():
        le_gender, le_season, le_usage, le_article = encoders[category]
        try:
            features = {
                "gender_enc": le_gender.transform([gender])[0],
                "season_enc": le_season.transform([weather["season"]])[0],
                "usage_enc": le_usage.transform([usage])[0],
                "temp_min": weather["feels_like"] - 5,
                "temp_max": weather["feels_like"] + 5
            }
            X_pred = pd.DataFrame([features])
            pred_enc = model.predict(X_pred)[0]
            predicted_article = le_article.inverse_transform([pred_enc])[0]


            category_df = df[(df["category"] == category) &
                             (df["articleType"] == predicted_article) &
                             (df["gender"] == gender) &
                             (df["usage"] == usage)]

            if not category_df.empty:
                recommended = category_df.sample(1).iloc[0]
                outfit[category] = {
                    "article": recommended["articleType"],
                    "color": recommended["baseColour"],
                    "product": recommended["productDisplayName"]
                }
            else:
                outfit[category] = None
        except Exception as e:
            print(f"Warning: Could not predict for {category}. Error: {str(e)}")
            outfit[category] = None
    return outfit


def get_personalized_recommendation(city, gender, usage):
    try:
        weather = get_weather(city)
        df = load_and_preprocess_data()
        models, encoders = train_rf_models(df)
        outfit = predict_outfit(models, encoders, df, weather, gender, usage)
        return weather, outfit
    except Exception as e:
        raise Exception(f"Recommendation failed: {str(e)}")

if __name__ == "__main__":
    print("Weather-Based Clothing Recommendation System")
    print("------------------------------------------")
    city = input("Enter your city: ").strip()
    gender = input("Enter your gender (Men/Women/Unisex): ").strip().lower()
    usage = input("Enter the usage type (e.g. Casual, Sports, Party): ").strip().title()
    try:
        weather, outfit = get_personalized_recommendation(city, gender, usage)
        print("\nPersonalized Clothing Recommendations:")
        print(f"Based on weather in {weather['season']} (Feels like: {weather['feels_like']}°C) for {usage} use")
        for category, details in outfit.items():
            if details:
                print(f"\n{category}:")
                print(f"- Type: {details['article']}")
                print(f"- Color: {details['color']}")
                print(f"- Product: {details['product']}")
            else:
                print(f"\n{category}: No suitable recommendation found")
    except Exception as e:
        print(f"\nError: {str(e)}")


Weather-Based Clothing Recommendation System
------------------------------------------
Enter your city: Bursa
Enter your gender (Men/Women/Unisex): Men
Enter the usage type (e.g. Casual, Sports, Party): Casual

Personalized Clothing Recommendations:
Based on weather in Spring (Feels like: 9.0°C) for Casual use

Topwear:
- Type: Tshirts
- Color: Red
- Product: Manchester United Men Printed Red Tshirt

Bottomwear:
- Type: Jeans
- Color: Blue
- Product: SPYKAR Men Blue Slim Fit Jeans

Footwear:
- Type: Casual Shoes
- Color: Blue
- Product: Gas Men Skate Blue Shoes


In [2]:
!pip install gradio

import gradio as gr
import pandas as pd
import requests
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# --- API KEY ve URL değil, gerçek hava durumunu kendi fonksiyonla çekiyoruz ---
API_KEY = "16ce454570964d39901204010250905"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather(city):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&lang=en"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Weather data could not be fetched: {response.status_code}, {response.text}")

    data = response.json()
    current = data["current"]
    location = data["location"]

    month = datetime.strptime(location['localtime'], "%Y-%m-%d %H:%M").month
    season = convert_to_season(month)

    return {
        "season": season,
        "temp": current["temp_c"],
        "feels_like": current["feelslike_c"],
        "humidity": current["humidity"],
        "wind_speed": current["wind_kph"],
        "condition": current["condition"]["text"].lower()
    }

def convert_to_season(month):
    if month in [12, 1, 2]: return "Winter"
    elif month in [3, 4, 5]: return "Spring"
    elif month in [6, 7, 8]: return "Summer"
    else: return "Fall"

def load_and_preprocess_data():
    df = pd.read_csv("styles_temizlenmis_kategoriler.csv", on_bad_lines='skip')
    df = df.drop(columns=[col for col in df.columns if col.startswith("Unnamed")])
    df = df[["id", "gender", "masterCategory", "subCategory", "articleType",
             "baseColour", "season", "usage", "productDisplayName"]].dropna().copy()

    df["gender"] = df["gender"].str.strip().str.lower()
    df["season"] = df["season"].str.strip().str.capitalize()
    df["subCategory"] = df["subCategory"].astype(str).str.strip().str.title()
    df["masterCategory"] = df["masterCategory"].astype(str).str.strip().str.title()
    df["articleType"] = df["articleType"].str.strip().str.title()
    df["usage"] = df["usage"].str.strip().str.title()

    valid_master = ["Apparel", "Accessories", "Footwear", "Personal Care", "Free Items", "Sporting Goods", "Home"]
    mask_invalid_master = ~df["masterCategory"].isin(valid_master) & df["subCategory"].isin(valid_master)
    df.loc[mask_invalid_master, ["masterCategory", "subCategory"]] = df.loc[mask_invalid_master, ["subCategory", "masterCategory"]].values

    df = df[df["gender"].isin(["men", "women", "unisex"])]
    df = df[df["season"].isin(["Summer", "Winter", "Spring", "Fall"])]

    def map_article_to_category(x):
        x = x.lower()
        if "shoe" in x or "sandal" in x or "flip flop" in x:
            return "Footwear"
        elif "shirt" in x or "tshirt" in x or "top" in x or "sweater" in x or "jacket" in x:
            return "Topwear"
        elif "jean" in x or "pant" in x or "trouser" in x or "short" in x or "skirt" in x or "legging" in x:
            return "Bottomwear"
        else:
            return None

    df["category"] = df["articleType"].apply(map_article_to_category)
    df = df.dropna(subset=["category"]).copy()

    def assign_weather_features(row):
        if row["season"] == "Winter": return {"temp_min": -5, "temp_max": 10}
        elif row["season"] == "Summer": return {"temp_min": 20, "temp_max": 40}
        else: return {"temp_min": 10, "temp_max": 25}

    weather_features = df.apply(assign_weather_features, axis=1, result_type='expand')
    df = pd.concat([df, weather_features], axis=1)
    return df

def train_rf_models(df):
    models, encoders = {}, {}
    for category in df["category"].unique():
        category_df = df[df["category"] == category].copy()
        le_gender, le_season, le_usage, le_article = LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()
        category_df["gender_enc"] = le_gender.fit_transform(category_df["gender"])
        category_df["season_enc"] = le_season.fit_transform(category_df["season"])
        category_df["usage_enc"] = le_usage.fit_transform(category_df["usage"])
        category_df["article_enc"] = le_article.fit_transform(category_df["articleType"])
        X = category_df[["gender_enc", "season_enc", "usage_enc", "temp_min", "temp_max"]]
        y = category_df["article_enc"]
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X, y)
        models[category] = model
        encoders[category] = (le_gender, le_season, le_usage, le_article)
    return models, encoders

def predict_outfit(models, encoders, df, weather, gender, usage):
    gender = gender.lower()
    usage = usage.title()
    outfit = {}
    for category, model in models.items():
        le_gender, le_season, le_usage, le_article = encoders[category]
        try:
            features = {
                "gender_enc": le_gender.transform([gender])[0],
                "season_enc": le_season.transform([weather["season"]])[0],
                "usage_enc": le_usage.transform([usage])[0],
                "temp_min": weather["feels_like"] - 5,
                "temp_max": weather["feels_like"] + 5
            }
            X_pred = pd.DataFrame([features])
            pred_enc = model.predict(X_pred)[0]
            predicted_article = le_article.inverse_transform([pred_enc])[0]

            category_df = df[(df["category"] == category) &
                             (df["articleType"] == predicted_article) &
                             (df["gender"] == gender) &
                             (df["usage"] == usage)]

            if not category_df.empty:
                recommended = category_df.sample(1).iloc[0]
                outfit[category] = {
                    "article": recommended["articleType"],
                    "color": recommended["baseColour"],
                    "product": recommended["productDisplayName"]
                }
            else:
                outfit[category] = None
        except Exception as e:
            outfit[category] = None
    return outfit

def get_personalized_recommendation(city, gender, usage):
    weather = get_weather(city)
    df = load_and_preprocess_data()
    models, encoders = train_rf_models(df)
    outfit = predict_outfit(models, encoders, df, weather, gender, usage)
    return weather, outfit

# Gradio arayüz fonksiyonu
def get_recommendation(city, gender, usage):
    try:
        weather, outfit = get_personalized_recommendation(city, gender, usage)
        result = f"Weather Info for {city} ({weather['season']})\n"
        result += f"Feels like: {weather['feels_like']}°C\n"
        result += f"Usage: {usage}\n\n"

        for category, details in outfit.items():
            result += f"{category}:\n"
            if details:
                result += f"  - Type: {details['article']}\n"
                result += f"  - Color: {details['color']}\n"
                result += f"  - Product: {details['product']}\n"
            else:
                result += "  No suitable recommendation found\n"
            result += "\n"
        return result
    except Exception as e:
        return f"Error: {str(e)}"


demo = gr.Interface(
    fn=get_recommendation,
    inputs=[
        gr.Textbox(label="Enter your city"),
        gr.Dropdown(choices=["men", "women", "unisex"], label="Select your gender"),
        gr.Dropdown(choices=["Casual", "Sports", "Party"], label="Select usage type")
    ],
    outputs="text",
    title="Weather-Based Clothing Recommendation"
)

demo.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://020ef20971f9160a11.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory